### baseline 코드
- 3layer 
- xavier init
- Adam
- training_epochs = 55
- hidden layer node 개수: 256

-> Accuracy: 0.725806474685669


## 위의 베이스라인 코드에 scaler를 사용해서 정확도를 높였습니다.

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/city-commercialchange-analysis/train.csv
/kaggle/input/city-commercialchange-analysis/submit.csv
/kaggle/input/city-commercialchange-analysis/test.csv


In [11]:
import torch
import torchvision.datasets as data
import torchvision.transforms as transforms
import random

In [2]:
import numpy as np
import torch
import torch.optim as optim
import pandas as pd

xy=pd.read_csv('/kaggle/input/city-commercialchange-analysis/train.csv')
xy

,year,quarter,city,Operating months,Closing months,Operating months_Average,Closing months_Average,Commercial change
0,2016,1,11290,99,51,1.031250,1.062500,0
1,2019,4,11290,111,56,1.009091,1.037037,0
2,2015,4,11380,92,48,0.968421,1.000000,2
3,2019,4,11200,105,57,0.954545,1.055556,2
4,2014,2,11260,91,42,0.989130,0.933333,3
...,...,...,...,...,...,...,...,...
546,2017,2,11350,91,48,0.938144,0.960000,3
547,2019,4,11500,94,50,0.854545,0.925926,3
548,2016,3,11200,95,51,0.989583,1.040816,2
549,2018,1,11620,99,50,0.990000,1.000000,3


In [3]:
corr=xy.corr(method='pearson')
corr

,year,quarter,city,Operating months,Closing months,Operating months_Average,Closing months_Average,Commercial change
year,1.000000,-0.011769,0.012068,0.467735,0.685062,-0.018703,-0.024846,-0.010037
quarter,-0.011769,1.000000,0.006998,0.070415,0.098112,0.020046,-0.024104,-0.005533
city,0.012068,0.006998,1.000000,-0.416362,-0.401390,-0.490152,-0.573218,0.387811
Operating months,0.467735,0.070415,-0.416362,1.000000,0.842963,0.850369,0.699022,-0.712853
Closing months,0.685062,0.098112,-0.401390,0.842963,1.000000,0.575777,0.687939,-0.575557
Operating months_Average,-0.018703,0.020046,-0.490152,0.850369,0.575777,1.000000,0.828461,-0.814984
Closing months_Average,-0.024846,-0.024104,-0.573218,0.699022,0.687939,0.828461,1.000000,-0.801290
Commercial change,-0.010037,-0.005533,0.387811,-0.712853,-0.575557,-0.814984,-0.801290,1.000000


In [4]:
x_data=xy.iloc[:,0:7]    #0~7 col
y_data=xy.iloc[:,7]

x_data

,year,quarter,city,Operating months,Closing months,Operating months_Average,Closing months_Average
0,2016,1,11290,99,51,1.031250,1.062500
1,2019,4,11290,111,56,1.009091,1.037037
2,2015,4,11380,92,48,0.968421,1.000000
3,2019,4,11200,105,57,0.954545,1.055556
4,2014,2,11260,91,42,0.989130,0.933333
...,...,...,...,...,...,...,...
546,2017,2,11350,91,48,0.938144,0.960000
547,2019,4,11500,94,50,0.854545,0.925926
548,2016,3,11200,95,51,0.989583,1.040816
549,2018,1,11620,99,50,0.990000,1.000000


In [5]:
y_data

0      0
1      0
2      2
3      2
4      3
      ..
546    3
547    3
548    2
549    3
550    0
Name: Commercial change, Length: 551, dtype: int64

In [6]:
x_train=np.array(x_data)
y_train=np.array(y_data)

x_train=torch.FloatTensor(x_train)
y_train=torch.LongTensor(y_train)

x_train[:5]

tensor([[2.0160e+03, 1.0000e+00, 1.1290e+04, 9.9000e+01, 5.1000e+01, 1.0312e+00,
         1.0625e+00],
        [2.0190e+03, 4.0000e+00, 1.1290e+04, 1.1100e+02, 5.6000e+01, 1.0091e+00,
         1.0370e+00],
        [2.0150e+03, 4.0000e+00, 1.1380e+04, 9.2000e+01, 4.8000e+01, 9.6842e-01,
         1.0000e+00],
        [2.0190e+03, 4.0000e+00, 1.1200e+04, 1.0500e+02, 5.7000e+01, 9.5455e-01,
         1.0556e+00],
        [2.0140e+03, 2.0000e+00, 1.1260e+04, 9.1000e+01, 4.2000e+01, 9.8913e-01,
         9.3333e-01]])

In [7]:
x_train.shape

torch.Size([551, 7])

In [8]:
y_train.shape

torch.Size([551])

In [9]:
y_train

tensor([0, 0, 2, 2, 3, 3, 3, 3, 1, 3, 3, 3, 3, 3, 0, 3, 3, 3, 3, 2, 3, 0, 3, 3,
        3, 0, 0, 0, 2, 1, 0, 0, 3, 3, 3, 3, 0, 0, 0, 3, 3, 3, 0, 3, 3, 0, 0, 3,
        3, 0, 0, 3, 0, 3, 0, 0, 0, 2, 3, 1, 3, 1, 0, 0, 0, 1, 0, 0, 3, 3, 1, 3,
        0, 3, 3, 0, 0, 3, 3, 3, 3, 3, 1, 3, 3, 1, 3, 0, 3, 3, 3, 0, 3, 3, 3, 0,
        3, 0, 0, 0, 3, 3, 2, 3, 3, 3, 3, 3, 3, 3, 3, 0, 3, 1, 3, 3, 3, 0, 3, 3,
        3, 0, 2, 2, 3, 3, 0, 3, 1, 0, 2, 0, 1, 3, 3, 3, 3, 3, 3, 3, 1, 3, 1, 3,
        3, 0, 3, 1, 2, 3, 3, 3, 3, 3, 0, 0, 3, 3, 0, 3, 3, 3, 2, 0, 1, 3, 3, 0,
        3, 3, 3, 1, 0, 0, 3, 0, 3, 3, 3, 3, 3, 3, 0, 3, 0, 3, 3, 3, 3, 3, 3, 0,
        3, 0, 2, 3, 0, 0, 2, 3, 3, 0, 0, 3, 2, 3, 0, 0, 3, 3, 2, 2, 3, 2, 3, 3,
        3, 0, 0, 2, 0, 3, 0, 3, 3, 1, 3, 0, 3, 3, 3, 1, 0, 0, 1, 1, 0, 3, 3, 0,
        2, 2, 0, 3, 3, 3, 0, 3, 0, 0, 1, 3, 3, 0, 0, 3, 0, 0, 1, 3, 1, 1, 0, 3,
        3, 1, 2, 0, 3, 3, 3, 3, 3, 0, 3, 2, 3, 0, 3, 3, 3, 0, 0, 3, 0, 3, 3, 3,
        3, 0, 3, 3, 3, 3, 3, 3, 1, 0, 0,

In [12]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

random.seed(777)
torch.manual_seed(777)
if device == 'cuda':
  torch.cuda.manual_seed_all(777)

In [13]:
# 학습 파라미터 설정
learning_rate = 0.01
training_epochs = 55
batch_size = 100

from sklearn import preprocessing
Scaler = preprocessing.StandardScaler()  

In [14]:
x_train

tensor([[2.0160e+03, 1.0000e+00, 1.1290e+04,  ..., 5.1000e+01, 1.0312e+00,
         1.0625e+00],
        [2.0190e+03, 4.0000e+00, 1.1290e+04,  ..., 5.6000e+01, 1.0091e+00,
         1.0370e+00],
        [2.0150e+03, 4.0000e+00, 1.1380e+04,  ..., 4.8000e+01, 9.6842e-01,
         1.0000e+00],
        ...,
        [2.0160e+03, 3.0000e+00, 1.1200e+04,  ..., 5.1000e+01, 9.8958e-01,
         1.0408e+00],
        [2.0180e+03, 1.0000e+00, 1.1620e+04,  ..., 5.0000e+01, 9.9000e-01,
         1.0000e+00],
        [2.0140e+03, 3.0000e+00, 1.1200e+04,  ..., 4.7000e+01, 1.0108e+00,
         1.0444e+00]])

In [15]:
x_train_scaler=Scaler.fit_transform(x_train)
x_train_scaler

array([[-0.32987277, -1.35424363, -0.70042932, ...,  0.50786107,
         0.35764194,  1.16085191],
       [ 1.40117387,  1.3428223 , -0.70042932, ...,  1.84933493,
         0.11025825,  0.69036582],
       [-0.90688832,  1.3428223 , -0.21609145, ..., -0.29702325,
        -0.34377913,  0.00602341],
       ...,
       [-0.32987277,  0.44380032, -1.18476718, ...,  0.50786107,
        -0.10752401,  0.76019677],
       [ 0.82415832, -1.35424363,  1.07547618, ...,  0.23956629,
        -0.10287202,  0.00602341],
       [-1.48390387,  0.44380032, -1.18476718, ..., -0.56531802,
         0.1288103 ,  0.82723474]])

In [16]:
x_train_scaler=torch.FloatTensor(x_train_scaler)

In [17]:

train = torch.utils.data.TensorDataset(x_train_scaler, y_train)

In [18]:
data_loader = torch.utils.data.DataLoader(dataset=train,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True)
#xy=train

In [19]:
# 3-Layer

linear1 = torch.nn.Linear(7,256,bias=True)
linear2 = torch.nn.Linear(256,256,bias=True)
linear3 = torch.nn.Linear(256,4,bias=True)
relu = torch.nn.ReLU()

In [20]:
# Random Init => Xavier Init
torch.nn.init.xavier_normal_(linear1.weight)
torch.nn.init.xavier_normal_(linear2.weight)
torch.nn.init.xavier_normal_(linear3.weight)

Parameter containing:
tensor([[ 0.0191,  0.0337,  0.0226,  ..., -0.0312, -0.0489, -0.0005],
        [-0.0130,  0.0307, -0.0692,  ..., -0.0873, -0.1966,  0.0183],
        [-0.0366,  0.0039, -0.0539,  ...,  0.1732, -0.0786, -0.0262],
        [-0.0586, -0.1101, -0.1071,  ..., -0.0391,  0.0904, -0.0184]],
       requires_grad=True)

In [21]:
model = torch.nn.Sequential(linear1,relu,linear2,relu,linear3).to(device)

In [22]:
# 손실함수와 최적화 함수
loss = torch.nn.CrossEntropyLoss().to(device) # softmax 내부적으로 계산
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate) 

In [23]:

total_batch = len(data_loader)
for epoch in range(training_epochs):
    avg_cost = 0

    for X, Y in data_loader:

        # one-hot encoding되어 있지 않음
        Y = Y.to(device)
        #%debug

        # 그래디언트 초기화
        optimizer.zero_grad()
        # Forward 계산
        hypothesis = model(X)
        # Error 계산
        cost = loss(hypothesis, Y)
        # Backparopagation
        cost.backward()
        # 가중치 갱신
        optimizer.step()

        # 평균 Error 계산
        avg_cost += cost / total_batch

    print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost))

print('Learning finished')

Epoch: 0001 cost = 0.740128756
Epoch: 0002 cost = 0.236988217
Epoch: 0003 cost = 0.178716734
Epoch: 0004 cost = 0.096945584
Epoch: 0005 cost = 0.118398279
Epoch: 0006 cost = 0.073740423
Epoch: 0007 cost = 0.062091969
Epoch: 0008 cost = 0.064697370
Epoch: 0009 cost = 0.050925933
Epoch: 0010 cost = 0.042971738
Epoch: 0011 cost = 0.030599557
Epoch: 0012 cost = 0.029026011
Epoch: 0013 cost = 0.023950150
Epoch: 0014 cost = 0.028708436
Epoch: 0015 cost = 0.021380568
Epoch: 0016 cost = 0.030182401
Epoch: 0017 cost = 0.012653408
Epoch: 0018 cost = 0.011398055
Epoch: 0019 cost = 0.015973808
Epoch: 0020 cost = 0.008835332
Epoch: 0021 cost = 0.010934482
Epoch: 0022 cost = 0.011986743
Epoch: 0023 cost = 0.012434644
Epoch: 0024 cost = 0.023043530
Epoch: 0025 cost = 0.021049971
Epoch: 0026 cost = 0.020623880
Epoch: 0027 cost = 0.008888572
Epoch: 0028 cost = 0.012067920
Epoch: 0029 cost = 0.012571150
Epoch: 0030 cost = 0.010757684
Epoch: 0031 cost = 0.007689484
Epoch: 0032 cost = 0.007694867
Epoch: 0

In [25]:
test=pd.read_csv('/kaggle/input/city-commercialchange-analysis/test.csv')

with torch.no_grad():
  x_test=test.loc[:,:]
  x_test=np.array(x_test)
  x_test_scaler=Scaler.transform(x_test)
  x_test_scaler=torch.from_numpy(x_test_scaler).float().to(device)

  prediction=model(x_test_scaler)
  prediction = torch.argmax(prediction, 1)

prediction

tensor([3, 0, 0, 0, 3, 0, 0, 1, 3, 0, 0, 3, 3, 0, 3, 0, 3, 0, 0, 3, 0, 0, 0, 3,
        0, 0, 3, 3, 0, 0, 0, 3, 0, 3, 3, 3, 1, 0, 3, 3, 3, 1, 3, 3, 0, 3, 3, 3,
        3, 3, 0, 0, 3, 3, 3, 3, 3, 3, 3, 1, 3, 0])

In [26]:
submit = pd.read_csv('/kaggle/input/city-commercialchange-analysis/submit.csv')
submit

,ID,Label
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0
...,...,...
57,57,0
58,58,0
59,59,0
60,60,0


In [27]:
id=np.array([i for i in range(62)]).reshape(-1,1)
prediction=prediction.reshape(-1,1)

result=np.hstack([id, prediction])
df=pd.DataFrame(result, columns=['ID','Label'])
df.to_csv('defense_submit.csv', index=False, header=True)

result

array([[ 0,  3],
       [ 1,  0],
       [ 2,  0],
       [ 3,  0],
       [ 4,  3],
       [ 5,  0],
       [ 6,  0],
       [ 7,  1],
       [ 8,  3],
       [ 9,  0],
       [10,  0],
       [11,  3],
       [12,  3],
       [13,  0],
       [14,  3],
       [15,  0],
       [16,  3],
       [17,  0],
       [18,  0],
       [19,  3],
       [20,  0],
       [21,  0],
       [22,  0],
       [23,  3],
       [24,  0],
       [25,  0],
       [26,  3],
       [27,  3],
       [28,  0],
       [29,  0],
       [30,  0],
       [31,  3],
       [32,  0],
       [33,  3],
       [34,  3],
       [35,  3],
       [36,  1],
       [37,  0],
       [38,  3],
       [39,  3],
       [40,  3],
       [41,  1],
       [42,  3],
       [43,  3],
       [44,  0],
       [45,  3],
       [46,  3],
       [47,  3],
       [48,  3],
       [49,  3],
       [50,  0],
       [51,  0],
       [52,  3],
       [53,  3],
       [54,  3],
       [55,  3],
       [56,  3],
       [57,  3],
       [58,  3